In [2]:
import os
from PIL import Image
import cv2
import numpy as np
import torch
import torchvision.models as models
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from sklearn.metrics import precision_score, recall_score, roc_auc_score
import torch
from sklearn.metrics import precision_score, recall_score, accuracy_score, roc_auc_score
import matplotlib.pyplot as plt

In [3]:
# Step 2: Set up the device (CUDA if available)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [4]:
# Step 3: Custom Dataset Class to Load Images and Labels
class CustomImageDataset(torch.utils.data.Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        # Load the image using OpenCV
        img = cv2.imread(self.image_paths[idx])
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert from BGR to RGB
        
        # Convert the NumPy array to a PIL Image
        img = Image.fromarray(img)
        
        # Apply the transformations (resize, normalize, etc.)
        if self.transform:
            img = self.transform(img)
        
        # Get the label
        label = torch.tensor(self.labels[idx], dtype=torch.long)  # Ensure label is torch.long
        
        return img, label


In [5]:
# Step 4: Load the 1658 Colored Images 
def load_colored_images(dataset_path):
    image_paths = []
    labels = []
    
    # Ensure that class folders are correctly identified
    class_names = sorted([folder for folder in os.listdir(dataset_path) if os.path.isdir(os.path.join(dataset_path, folder))])  # Class folders (9 classes)
    
    # Loop through each class folder
    for class_index, class_name in enumerate(class_names):
        class_folder = os.path.join(dataset_path, class_name)
        
        # Loop through each image in the class folder
        for img_name in os.listdir(class_folder):
            img_path = os.path.join(class_folder, img_name)
            if img_name.lower().endswith(('.jpg', '.png', '.jpeg', '.bmp')):  # Check for valid image extensions
                image_paths.append(img_path)
                labels.append(class_index)  # Assign the class label
    
    return image_paths, np.array(labels), class_names

# Path to your dataset (colored images)
dataset_path = 'D:\\Desktop\\AI361\\project\\clean_resized'

# Load the image paths and labels for the 1658 colored images
image_paths, labels, class_names = load_colored_images(dataset_path)

# Output some useful information to verify the function worked
print(f"Loaded {len(image_paths)} images from {len(class_names)} classes:")
for class_name in class_names:
    print(f"Class '{class_name}' contains the following images:")
    class_image_paths = [image_paths[i] for i in range(len(labels)) if labels[i] == class_names.index(class_name)]
    print(f"  - {len(class_image_paths)} images")



Loaded 1658 images from 9 classes:
Class 'Ajwa' contains the following images:
  - 175 images
Class 'Galaxy' contains the following images:
  - 190 images
Class 'Medjool' contains the following images:
  - 135 images
Class 'Meneifi' contains the following images:
  - 232 images
Class 'Nabtat Ali' contains the following images:
  - 177 images
Class 'Rutab' contains the following images:
  - 146 images
Class 'Shaishe' contains the following images:
  - 171 images
Class 'Sokari' contains the following images:
  - 264 images
Class 'Sugaey' contains the following images:
  - 168 images


In [6]:
# Step 5: Data Augmentation and Normalization for Colored Images
transform = transforms.Compose([
    transforms.Resize((224, 224)),               # Resize image to 224x224
    transforms.ToTensor(),                       # Convert image to tensor and normalize to [0, 1]
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize for pre-trained models
])

# Create a dataset and data loader
dataset = CustomImageDataset(image_paths, labels, transform=transform)
train_loader = DataLoader(dataset, batch_size=64, shuffle=True)
#train_loader = DataLoader(
    #dataset, 
    #batch_size=32, 
    #shuffle=True, 
    #num_workers=4,  # Increase workers to speed up data loading
    #pin_memory=True)  # Optionally enable for faster data transfer to GPU




In [7]:
# Step 6: Load 600 Noisy Grayscale Images
def load_noisy_grayscale_images(noisy_images_path):
    noisy_images = []
    image_names = []  # To store image names
    
    for img_name in os.listdir(noisy_images_path):
        if img_name.endswith('.jpg') or img_name.endswith('.png'):
            img_path = os.path.join(noisy_images_path, img_name)
            
            # Read image in grayscale
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, (224, 224))  # Resize image to 224x224
            
            # Normalize image to [0, 1]
            img = img.astype('float32') / 255.0
            
            # Convert grayscale to 3 channels (RGB) by repeating the single channel
            img = np.expand_dims(img, axis=-1)  # Shape: (224, 224, 1)
            img = np.repeat(img, 3, axis=-1)    # Shape: (224, 224, 3)
            
            # Convert to Tensor and normalize
            img = transforms.ToTensor()(img)
            img = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])(img)
            
            noisy_images.append(img)
            image_names.append(img_name)  # Store the name of the image
    
    return torch.stack(noisy_images), image_names

# Path to the folder containing the 600 noisy grayscale images
noisy_images_path = 'D:\\Desktop\\AI361\\project\\enhanced-images31'

# Load noisy grayscale images
noisy_grayscale_images, noisy_image_names = load_noisy_grayscale_images(noisy_images_path)
print(f"Loaded {len(noisy_grayscale_images)} noisy grayscale images.")


Loaded 600 noisy grayscale images.


In [8]:
# Step 7: Define CNN Model for Classification
class ModifiedCNNModel(nn.Module):
    def __init__(self, num_classes):
        super(ModifiedCNNModel, self).__init__()
        
        # Use pre-trained MobileNetV2 as the backbone
        self.mobilenet_v2 = models.mobilenet_v2(pretrained=True).features
        
        # Freeze the layers of MobileNetV2
        for param in self.mobilenet_v2.parameters():
            param.requires_grad = False
        
        # Custom fully connected layers after the MobileNetV2 feature extractor
        # Update the input size of fc1 to 62720 (1280 * 7 * 7)
        self.fc1 = nn.Linear(1280 * 7 * 7, 256)  # MobileNetV2 output channels = 1280, spatial size = 7x7
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 32)
        self.fc5 = nn.Linear(32, 16)
        self.fc6 = nn.Linear(16, num_classes)
        
        # Dropout for regularization
        self.dropout = nn.Dropout(0.2)
    
    def forward(self, x):
        # Pass input through MobileNetV2 feature extractor
        x = self.mobilenet_v2(x)
        
        # Flatten the output for the fully connected layers
        x = x.view(x.size(0), -1)  # Flatten to (batch_size, 1280 * 7 * 7)
        
        # Fully connected layers with ReLU activations and Dropout
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = F.relu(self.fc3(x))
        x = self.dropout(x)
        x = F.relu(self.fc4(x))
        x = self.dropout(x)
        x = F.relu(self.fc5(x))
        x = self.dropout(x)
        
        # Final output layer
        x = self.fc6(x)
        
        return x

# Initialize the model
num_classes = 9  # Replace with the number of output classes
model = ModifiedCNNModel(num_classes=num_classes)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

print(model)


C:\Users\96658\anaconda3\envs\gpu_env\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\96658\anaconda3\envs\gpu_env\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ModifiedCNNModel(
  (mobilenet_v2): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d

In [ ]:
# Step 8: Set up Loss Function and Optimizer
# Define the model (assuming a custom model is defined like in your previous code)
# Initialize the model
num_classes = 9  # Replace with the number of output classes
model = ModifiedCNNModel(num_classes=num_classes)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Loss function and optimizer (same as in code 2)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Callback for early stopping (this is a manual implementation in PyTorch)
early_stopping_patience = 30
best_loss = float('inf')
epochs_without_improvement = 0

# Metrics to track (similar to the TensorFlow/Keras code)
from sklearn.metrics import precision_score, recall_score, roc_auc_score

def compute_metrics(outputs, labels, num_classes):
    # Apply softmax to get class probabilities
    probs = torch.softmax(outputs, dim=1)
    
    # Get the predicted class from probabilities
    predicted = torch.argmax(probs, dim=1)
    
    # Calculate accuracy
    accuracy = (predicted == labels).sum().item() / labels.size(0)
    
    # Calculate precision and recall (weighted average)
    precision = precision_score(labels.cpu(), predicted.cpu(), average='weighted', zero_division=0)
    recall = recall_score(labels.cpu(), predicted.cpu(), average='weighted', zero_division=0)
    
    # Calculate AUC for multi-class
    auc = float('nan')  # Initialize with NaN, as default
    if len(torch.unique(labels)) > 1:  # Only compute AUC if there are at least 2 unique classes in the batch
        try:
            # One-hot encode labels and calculate AUC
            one_hot_labels = torch.eye(num_classes)[labels.cpu()].numpy()
            auc = roc_auc_score(one_hot_labels, probs.cpu().detach().numpy(), multi_class='ovr', average='weighted')
        except ValueError as e:
            print(f"Error in AUC calculation: {e}")
            auc = float('nan')  # If AUC fails, set it to NaN
    
    return accuracy, precision, recall, auc




# Training loop
epochs = 300
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0
    
    # For each batch in the train_loader
    for inputs, labels in train_loader:
        # Move inputs and labels to GPU if available
        inputs, labels = inputs.to(device), labels.to(device)
    
        # Zero the gradients
        optimizer.zero_grad()
    
        # Forward pass
        outputs = model(inputs)
    
        # Compute loss
        loss = criterion(outputs, labels)
    
        # Backward pass
        loss.backward()
        optimizer.step()
    
        # Track statistics
        running_loss += loss.item()
        
        # Compute metrics
        accuracy, precision, recall, auc = compute_metrics(outputs, labels, num_classes)
        
        correct_predictions += (outputs.argmax(dim=1) == labels).sum().item()
        total_predictions += labels.size(0)
    
    # Calculate average loss and accuracy
    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = 100 * correct_predictions / total_predictions
    
    # Print training statistics for the current epoch
    print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%")
    print(f"Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, AUC: {auc:.4f}")

    # Early stopping logic (manual)
    if epoch_loss < best_loss:
        best_loss = epoch_loss
        epochs_without_improvement = 0
        # Save model checkpoint if needed
        # torch.save(model.state_dict(), 'best_model.pth')
    else:
        epochs_without_improvement += 1
        if epochs_without_improvement >= early_stopping_patience:
            print("Early stopping triggered!")
            break



C:\Users\96658\anaconda3\envs\gpu_env\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\96658\anaconda3\envs\gpu_env\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Error in AUC calculation: Only one class present in y_true. ROC AUC score is not defined in that case.
Epoch 1/300, Loss: 1.9841, Accuracy: 24.49%
Accuracy: 0.3793, Precision: 0.3645, Recall: 0.3793, AUC: 0.7325
Epoch 2/300, Loss: 1.4182, Accuracy: 46.74%
Accuracy: 0.5862, Precision: 0.6582, Recall: 0.5862, AUC: 0.9043
Error in AUC calculation: Only one class present in y_true. ROC AUC score is not defined in that case.
Epoch 3/300, Loss: 0.9497, Accuracy: 66.71%
Accuracy: 0.5690, Precision: 0.6838, Recall: 0.5690, AUC: nan
Epoch 4/300, Loss: 0.5232, Accuracy: 81.97%
Accuracy: 0.9483, Precision: 0.9562, Recall: 0.9483, AUC: 0.9973
Error in AUC calculation: Only one class present in y_true. ROC AUC score is not defined in that case.
Epoch 5/300, Loss: 0.4233, Accuracy: 86.25%
Accuracy: 0.8448, Precision: 0.8861, Recall: 0.8448, AUC: 0.9860
Error in AUC calculation: Only one class present in y_true. ROC AUC score is not defined in that case.
Epoch 6/300, Loss: 0.2681, Accuracy: 90.59%
Ac

In [ ]:
# Step 9: Classify Noisy Grayscale Images with the Trained Model
model.eval()  # Set the model to evaluation mode
noisy_grayscale_images = noisy_grayscale_images.to(device)

with torch.no_grad():
    outputs = model(noisy_grayscale_images)
    _, predicted_labels = torch.max(outputs, 1)

# Convert predicted labels to numpy array
predicted_labels = predicted_labels.cpu().numpy()

# Create a dictionary to store the images belonging to each class
class_image_map = {class_name: [] for class_name in class_names}

# Map the images to their predicted classes
for idx, label in enumerate(predicted_labels):
    class_image_map[class_names[label]].append(noisy_image_names[idx])

# Display the results
for class_name, images in class_image_map.items():
    print(f"Class: {class_name}")
    print(f"Images: {', '.join(images)}\n")
